In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pearsonr
from json import dump
from sklearn.model_selection import RepeatedKFold
from collections import defaultdict
from collections import OrderedDict


##  DATA LOADING  ##

# load parameters
df_params = pd.read_csv('data/parameters.csv', header=[0], index_col=0)
parameters_raw = np.array([[df_params.loc[i][j] for j in df_params.columns] for i in df_params.index])
y_scalers = [MinMaxScaler() for _ in range(parameters_raw.shape[1])]
parameters = np.zeros_like(parameters_raw)
for i in range(parameters_raw.shape[1]):
    parameters[:, i] = y_scalers[i].fit_transform(parameters_raw[:, i].reshape(-1, 1)).flatten()

# load identifiers (vertical headers)
identifiers = df_params.index.values

# load markers
df_markers = pd.read_csv('data/markers.csv', header=[0], index_col=0)
df_markers = df_markers.transpose()
print(df_markers.shape)
markers_raw = np.array([df_markers.loc[i].values[1:].astype(float) for i in df_markers.index])
x_scaler = MinMaxScaler()

# scale the markers (apply the scaler globally not per columns)
flattened_markers = markers_raw.ravel()
scaled_markers = x_scaler.fit_transform(flattened_markers.reshape(-1, 1)).ravel()
markers = scaled_markers.reshape(markers_raw.shape)

# extract traits
traits = np.array([df_markers.loc[i].values[0] for i in df_markers.index])
df_params['trait'] = traits
trait_scaler = MinMaxScaler()
traits = trait_scaler.fit_transform(traits.reshape(-1, 1)).flatten()
y_scalers.append(trait_scaler)
parameters = np.column_stack((parameters, traits))


# Generate folds - either from R analyses or generated in python
def generate_folds(load_folds=False, num_repeats=2, num_folds=5, seed=42):
    if load_folds:
        ####### Load R Folds #######
        df_fold = pd.read_csv('data/folds_bayes_lasso.csv', header=[0], index_col=0)
        folds = np.array(df_fold.iloc[:, 0])
        num_repeats = 1
        num_folds = 5
        data_folds = {}
        for fold in range(1, num_folds + 1):
            test_idx = np.where(folds == fold)[0]
            train_idx = np.where(folds != fold)[0]

            test_data = {
                'indices': test_idx,
                'identifiers': identifiers[test_idx],
                'parameters': parameters[test_idx],
                'markers': markers[test_idx],
                'trait': traits[test_idx],
            }

            train_data = {
                'indices': train_idx,
                'identifiers': identifiers[train_idx],
                'parameters': parameters[train_idx],
                'markers': markers[train_idx],
                'trait': traits[train_idx],
            }

            data_folds[fold] = {
                'train': train_data,
                'test': test_data
            }
    else:
        ####### Generate repeated K-Folds #######
        num_samples = len(identifiers)
        data_folds = {}
        fold_indices = pd.DataFrame(identifiers, columns=['identifier'])
        for repeat_num in range(1, num_repeats + 1):
            fold_indices[repeat_num] = np.nan
            

        rkf = RepeatedKFold(n_splits=num_folds, n_repeats=num_repeats, random_state=seed)

        for repeat, (train_index, test_index) in enumerate(rkf.split(identifiers), start=1):
            fold = (repeat - 1) % num_folds + 1
            repeat_num = (repeat - 1) // num_folds + 1

            test_data = {
                'indices': test_index,
                'identifiers': identifiers[test_index],
                'parameters': parameters[test_index],
                'markers': markers[test_index],
                'trait': traits[test_index],
            }

            train_data = {
                'indices': train_index,
                'identifiers': identifiers[train_index],
                'parameters': parameters[train_index],
                'markers': markers[train_index],
                'trait': traits[train_index],
            }

            data_folds[(repeat_num, fold)] = {
                'train': train_data,
                'test': test_data,
                'repeat': repeat_num,
            }
            
            fold_indices.loc[test_index, repeat_num] = fold

        fold_indices.to_csv('generated/fold_indices.csv', index=False)
    
    return data_folds


# Metrics
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    pearson, _ = pearsonr(y_true, y_pred)
    return mse, rmse, mae, r2, pearson

# Sort results
def sort_dict_recursively(d):
    if isinstance(d, dict):
        # Sort keys: integers before strings
        sorted_items = sorted(d.items(), key=lambda x: (isinstance(x[0], str), x[0]))
        return OrderedDict(
            (k, sort_dict_recursively(v)) for k, v in sorted_items
        )
    elif isinstance(d, list):
        # Apply the function recursively to each item in the list
        return [sort_dict_recursively(i) for i in d]
    else:
        # Return the value as is if it's neither a dictionary nor a list
        return d
    

def nested_defaultdict():
    return defaultdict(nested_defaultdict)

(136, 31714)


C:\Users\U108-N257\AppData\Local\Temp\ipykernel_36720\3332090209.py:27: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_markers = pd.read_csv('data/markers.csv', header=[0], index_col=0)


In [2]:
import warnings
import json
import numpy as np

# Machine Learning Models
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.cross_decomposition import PLSRegression
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression, BayesianRidge
import catboost as cb

# Deep Learning Models

from tensorflow_utils import cnn_softmax, cnn_bacon, cnn_decon, CNN_transformer, depth_res, transformer_model
from tensorflow_utils import Auto_Save, clr
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler


# Suppress specific warning from PLS
warnings.filterwarnings("ignore", message="Y residual is constant at iteration")

# the list of models to be trained
ml_models = [
    ('knn', KNeighborsRegressor(leaf_size= 71, n_neighbors= 9, weights= 'distance', p= 2)),
    ('RF', RandomForestRegressor(bootstrap=False, max_depth=87, max_features="sqrt", min_samples_leaf=2, min_samples_split=6, n_estimators=318)),
    ('svm', SVR(kernel='rbf', epsilon=0.02604615195102571, C=1)),
    ('ElasticNet', ElasticNet(alpha=5.09563321426854, l1_ratio=2.5128617959953626e-08, max_iter=20000, tol=0.06185856629377155, selection='cyclic')),
    ('PLS', PLSRegression(n_components=100)),
    ('BayesianRidge', BayesianRidge(max_iter=1000, tol=0.0001)),
    ('Ridge', Ridge(alpha=82.61743872370975, solver='saga', max_iter=20000, tol=0.02531618477469109)),
    ('Lasso', Lasso(alpha=0.0017742588786429565, tol=0.002395379538656458, max_iter=40000)),
    ('XGBoost', XGBRegressor(n_estimators=970, max_depth=21, min_child_weight=3.8901787266768046, reg_lambda=1.2476564637797165, alpha=0.00015777908963872925, subsample=0.514174538559911, colsample_bytree=0.8748701490064003, eta=0.9436165548239603, gamma=0.050764442960315445, learning_rate=0.0003697533172127718, booster='gblinear', verbosity=0)),
    ('cnn_softmax', cnn_softmax(markers.shape[1]), 100),
    ('bacon', cnn_bacon(markers.shape[1]), 100),
    # ('AdaBoost', AdaBoostRegressor(n_estimators=55, learning_rate=0.00465096494044598, loss='exponential')),
    # ('CatBoost', cb.CatBoostRegressor(learning_rate=0.016588626830975256, depth=6, subsample=0.33380061672814154, colsample_bylevel=0.5087556106888949, min_data_in_leaf=35, silent=True)),
    
    # ('CNN_transformer', CNN_transformer(markers.shape[1]), 100),
    # ('decon', cnn_decon(markers.shape[1]), 100),
    # # ('depth_res', depth_res(markers.shape[1]), 50),
    # # ('transformer', transformer_model(markers.shape[1]), 20),
]


num_repeats = 15
num_folds = 5
load_folds = False # Set to True to load R folds, False to generate new folds with num_repeats and num_folds
seed = 42

# PREPARE DATA
data_folds = generate_folds(load_folds, num_repeats, num_folds, seed)

fold_indices = []
if load_folds:
    fold_indices = [(fold, str(fold)) for fold in range(1, num_folds + 1)]
else:
    fold_indices = [((repeat, fold), f"{repeat}_{fold}") for repeat in range(1, num_repeats + 1) for fold in range(1, num_folds + 1)]


results = {}
all_results_in_one = nested_defaultdict()
all_results_in_one["seed"] = seed


# TRAIN MODELS
for fold, foldname in fold_indices:
    train_data = data_folds[fold]['train']
    test_data = data_folds[fold]['test']

    train_identifiers = train_data['identifiers']
    train_parameters = train_data['parameters']
    train_markers = train_data['markers']
    train_trait = train_data['trait']

    test_identifiers = test_data['identifiers']
    test_parameters = test_data['parameters']
    test_markers = test_data['markers']
    test_trait = test_data['trait']

    if fold not in results:
        results[fold] = {}

    for config in ml_models:
        model_name = config[0]
        model = config[1]
        patience = config[2] if len(config) > 2 else None
        
        
        if model_name not in results[fold]:
            results[fold][model_name] = {}

        for i in range(train_parameters.shape[1]):
            param_key = df_params.columns[i]
            y = train_parameters[:, i]
            
            print(f'Fold {foldname} - {model_name} - {param_key} >', end=' ')
            
            if param_key not in results[fold][model_name]:
                results[fold][model_name][param_key] = {}

            if patience is None:
                model.fit(train_markers, y)
                predictions = model.predict(test_markers).flatten()
            else:
                reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=patience)
                early_stop = EarlyStopping(monitor="val_loss", patience=patience*3, verbose=0, mode="min", min_delta=0)
                auto_save = Auto_Save(model_name + "_" + param_key, verbose=0)
                lrScheduler = LearningRateScheduler(clr)
                model.fit(train_markers, y, epochs=patience*10, verbose=0, callbacks=[lrScheduler, early_stop, auto_save], validation_data=(test_markers, test_parameters[:, i]))
                predictions = model.predict(test_markers, verbose=0).flatten()
                
            
            # Compute metrics
            mse, rmse, mae, r2, pearson = calculate_metrics(test_parameters[:, i], predictions)
            unscaled_preds = y_scalers[i].inverse_transform(predictions.reshape(-1, 1)).flatten()
            unscaled_true = y_scalers[i].inverse_transform(test_parameters[:, i].reshape(-1, 1)).flatten()
            _, rmse_unscaled, _, _, _ = calculate_metrics(unscaled_true, unscaled_preds)


            results[fold][model_name][param_key] = {
                'RMSE': rmse,
                'RMSE_unscaled': rmse_unscaled,
                'MAE': mae,
                'MSE': mse,
                'R2': r2,
                'pearson': pearson,
                'predictions': unscaled_preds,
            }            
            print(f' RMSE: {rmse:.4f} - RMSE_unscaled: {rmse_unscaled:.4f} - MAE: {mae:.4f} - R2: {r2:.4f} - Pearson: {pearson:.4f}')


Fold 1_1 - bacon - Epsib > 

KeyboardInterrupt: 

In [131]:

# GENERATE REPORTS
keys = ['RMSE_unscaled', 'MAE', 'R2', 'pearson']
param_count = len(df_params.columns)

repeated_metrics = np.full((param_count + 4, len(ml_models) * len(keys) * num_repeats * num_folds + 1), np.nan).tolist()
for model_index, model_tuple in enumerate(ml_models):
    model_name = model_tuple[0]
    for key_index, key in enumerate(keys):
        for repeat_index in range(num_repeats):
            for fold_index in range(num_folds):
                repeated_metrics[0][ (key_index * len(ml_models) * num_repeats * num_folds) + (repeat_index * len(ml_models) * num_folds) + (fold_index * len(ml_models)) + model_index + 1 ] = key
                repeated_metrics[1][ (key_index * len(ml_models) * num_repeats * num_folds) + (repeat_index * len(ml_models) * num_folds) + (fold_index * len(ml_models)) + model_index + 1 ] = f'Repeat {repeat_index + 1}'
                repeated_metrics[2][ (key_index * len(ml_models) * num_repeats * num_folds) + (repeat_index * len(ml_models) * num_folds) + (fold_index * len(ml_models)) + model_index + 1 ] = f'Fold {fold_index + 1}'
                repeated_metrics[3][ (key_index * len(ml_models) * num_repeats * num_folds) + (repeat_index * len(ml_models) * num_folds) + (fold_index * len(ml_models)) + model_index + 1 ] = model_name
for param_index, param_key in enumerate(df_params.columns):
    repeated_metrics[param_index + 4][0] = param_key
    

repeated_predictions = np.full((len(identifiers) + 3, len(ml_models) * param_count * num_repeats + 1), np.nan).tolist()
for model_index, model_tuple in enumerate(ml_models):
    model_name = model_tuple[0]
    for param_index, param_key in enumerate(df_params.columns):
        for repeat_index in range(num_repeats):
            repeated_predictions[0][model_index * param_count * num_repeats + param_index * num_repeats + repeat_index + 1] = model_name
            repeated_predictions[1][model_index * param_count * num_repeats + param_index * num_repeats + repeat_index + 1] = param_key
            repeated_predictions[2][model_index * param_count * num_repeats + param_index * num_repeats + repeat_index + 1] = f'Repeat {repeat_index + 1}'
for idx, id in enumerate(identifiers):
    repeated_predictions[idx + 3][0] = id


mean_metrics = np.full((param_count + 2, len(ml_models) * len(keys) + 1), 0).tolist()
for model_index, model_tuple in enumerate(ml_models):
    model_name = model_tuple[0]
    for key_index, key in enumerate(keys):
        mean_metrics[0][key_index * len(ml_models) + model_index + 1] = key
        mean_metrics[1][key_index * len(ml_models) + model_index + 1] = model_name
for param_index, param_key in enumerate(df_params.columns):
    mean_metrics[param_index + 2][0] = param_key


mean_predictions = np.full((len(identifiers) + 2, len(ml_models) * param_count + 1), 0).tolist()
for model_index, model_tuple in enumerate(ml_models):
    model_name = model_tuple[0]
    for param_index, param_key in enumerate(df_params.columns):
        mean_predictions[0][param_index * len(ml_models) + model_index + 1] = param_key
        mean_predictions[1][param_index * len(ml_models) + model_index + 1] = model_name
for idx, id in enumerate(identifiers):
    mean_predictions[idx + 2][0] = id

# FILL REPORTS
for fold, fold_results in results.items():
    repeat_index = fold[0] if isinstance(fold, tuple) else 0
    fold_index = fold[1] if isinstance(fold, tuple) else fold
    sample_indices = data_folds[(repeat_index, fold_index)]['test']['indices']
    prediction_indices = sample_indices
    prediction_indices = prediction_indices.tolist()
    fold_index = fold_index - 1
    repeat_index = repeat_index - 1
    all_results_in_one["indices"][repeat_index][fold_index] = sample_indices.tolist()
    
    for model_index, (model_name, model_results) in enumerate(fold_results.items()):
        for param_index, (param_key, param_results) in enumerate(model_results.items()):
            for key_index, key in enumerate(keys):
                repeated_metrics[param_index + 4][(key_index * len(ml_models) * num_repeats * num_folds) + (repeat_index * len(ml_models) * num_folds) + (fold_index * len(ml_models)) + model_index + 1] = param_results[key]
                mean_metrics[param_index + 2][(key_index * len(ml_models)) + model_index + 1] += param_results[key] / (num_folds * num_repeats)
                all_results_in_one[model_name][param_key][key][repeat_index][fold_index] = param_results[key]
                all_results_in_one[model_name][param_key][key]["mean"] = mean_metrics[param_index + 2][(key_index * len(ml_models)) + model_index + 1]
                
            for row, src_index in enumerate(prediction_indices):
                repeated_predictions[src_index + 3][model_index * param_count * num_repeats + param_index * num_repeats + repeat_index + 1] = param_results['predictions'][row]
                mean_predictions[src_index + 2][param_index * len(ml_models) + model_index + 1] += param_results['predictions'][row] / num_repeats
                
                all_results_in_one[model_name][param_key]["predictions"][repeat_index][identifiers[src_index]] = param_results['predictions'][row]
                all_results_in_one[model_name][param_key]["predictions_mean"][identifiers[src_index]] = mean_predictions[src_index + 2][param_index * len(ml_models) + model_index + 1]

csv_predictions = pd.DataFrame(repeated_predictions)
csv_predictions.to_csv('reports/predictions.csv', index=False, header=False)

csv_metrics = pd.DataFrame(repeated_metrics)
csv_metrics.to_csv('reports/metrics.csv', index=False, header=False)

csv_mean_predictions = pd.DataFrame(mean_predictions)
csv_mean_predictions.to_csv('reports/mean_predictions.csv', index=False, header=False)

csv_mean_metrics = pd.DataFrame(mean_metrics)
csv_mean_metrics.to_csv('reports/mean_metrics.csv', index=False, header=False)

all_results_in_one = sort_dict_recursively(all_results_in_one)
json.dump(all_results_in_one, open('reports/all_results_in_one.json', 'w'), indent=4)


def convert_to_serializable(obj):
    if isinstance(obj, np.float32):
        return float(obj)
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")


# all_results_in_one = json.dumps(all_results_in_one, default=convert_to_serializable)
# all_results_in_one = sort_dict_recursively(all_results_in_one)
# # print(all_results_in_one)
# json.dump(all_results_in_one, open('all_results_in_one.json', 'w'), indent=4)